# Tahoe Forest Health Threshold Analysis
* Mason Bindl, mbindl@trpa.gov
* Andrew McClary, amcclary@trpa.gov

##### Links
* https://caregionalresourcekits.org//sierra.html#forest_res

### Setup

In [ ]:
# import packages
from utils import *
import os
import arcpy
from arcpy.sa import *
# from arcpy.ia import *
# from arcpy.ddd import *
import pandas as pd
from pathlib import Path

# setup workspace variables
arcpy.env.workspace = r"F:\\GIS\\PROJECTS\\ForestHealth_Intiative\\ThresholdUpdate\\Data\\ForestHealth_ThresholdUpdate.gdb"
arcpy.env.overwriteOutput = True

#format paths
sdeBase   = Path("F:\GIS\DB_CONNECT\Vector.sde")
# workspace = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb")
downloads = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Download\SNV_RRK")
output    = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct")

# base layers
trpa_boundary       = sdeBase / "SDE.Jurisdictions\SDE.TRPA_bdy"
mgmt_areas          = sdeBase / "SDE.Jurisdictions\SDE.ForestManagementZone_USFS"

# raw raster data downloaded from SNVRRK
veg_type            = downloads / "CWHR_vegetation\RRK_Fveg_WHRtype_2023Apr_4regions_v2.tif"
stand_density       = downloads / "TPA_2021_30m\TPA_2021_30m.tif"
basal_area          = downloads / "BASATOT_2021_30m\BASATOT_2021_30m.tif"
seral_stage         = downloads / "SeralStage_EML_2021\SeralStage_EML_2021.tif"
canopy_cover        = downloads / "CFO_CanopyCover2020Summer\CFO_CanopyCover2020Summer.tif"
fire_prob_low       = downloads / "ProbabilityLowFireSev_202308\ProbabilityLowFireSev_202308.tif"
fire_prob_moderate  = downloads / "ProbabilityModerateFireSev_202308\ProbabilityModerateFireSev_202308.tif"
fire_prob_high      = downloads / "ProbabilityHighFireSev_202308\ProbabilityHighFireSev_202308.tif"

In [ ]:
# list datasets in workspace
feature_data = arcpy.ListFeatureClasses()
raster_data  = arcpy.ListRasters()
table_data   = arcpy.ListTables()
# print feature classes, then rasters, then tables
print("Feature Classes:")
for fc in feature_data:
    print(fc)
print("\nRasters:")
for raster in raster_data:
    print(raster)
print("\nTables:")
for table in table_data:
    print(table)

## Extract to Tahoe

In [ ]:
# save a verions of each input tif at full extent and extract the extent to Tahoe
# create a list of the rasters to be clipped
raster_list = [
    veg_type,
    stand_density,
    basal_area,
    seral_stage,
    canopy_cover,
    fire_prob_low,
    fire_prob_moderate,
    fire_prob_high
]
# names of the rasters to be clipped
raster_names = [
    "veg_type_sierra",
    "stand_density_sierra",
    "basal_area_sierra",
    "seral_stage_sierra",
    "canopy_cover_sierra",
    "fire_prob_low_sierra",
    "fire_prob_moderate_sierra",
    "fire_prob_high_sierra"
]

# save a version of each input tif at full extent
for raster, name in zip(raster_list, raster_names):
    # save a version of each input tif at full extent
    out_raster = f"{name}"
    arcpy.sa.Raster(str(raster)).save(out_raster)
    print(f"Saved {name} to full extent")
# extract by mask and add _tahoe to the name
for name in raster_names:
    # extract by mask
    # format output name to take off _sieera and add _tahoe
    output_name = name.replace("_sierra", "_tahoe")
    out_extract = extract_by_mask_to_tahoe_extent(name, output_name)
    print(f"Extracted {output_name} to Tahoe")

## Stand Density

In [ ]:
# Map WHRTYPE codes to forest type codes
veg_map = {
    "JPN": 1, "EPN": 1, "JUN": 1, "LPN": 1,   # Jeffrey Pine
    "SMC": 2, "WFR": 2,                       # White fir-mixed conifer
    "RFR": 3,                                 # Red fir
    "SCN": 4,                                 # Sub Alpine Conifer
    "ASP": 5                                  # Aspen
}

# Build reclassified forest type raster from veg_type_tahoe using attribute WHRTYPE
def build_forest_type_raster(input_raster):
    remap = RemapValue([[k, v] for k, v in veg_map.items()])
    return Reclassify(input_raster, "WHRTYPE", remap, "NODATA")

# Function to evaluate pixels against threshold targets
def assess_density_targets(forest_type_raster, seral_stage_raster, stand_density_raster, basal_area_raster):
    thresholds = {
        (1, 1): (200, 30),  # JP Early
        (1, 2): (70, 80),   # JP Mid
        (1, 3): (60, 100),  # JP Late
        (2, 1): (300, 40),  # WF Early
        (2, 2): (100, 150), # WF Mid
        (2, 3): (80, 200),  # WF Late
        (3, 1): (300, 50),  # RF Early
        (3, 2): (100, 250), # RF Mid
        (3, 3): (80, 350),  # RF Late
        (4, 1): (400, 60),  # Sub Alpine Conifer Early
        (4, 2): (150, 300), # Sub Alpine Conifer Mid
        (4, 3): (120, 450), # Sub Alpine Conifer Late
        (5, 1): (400, 60),  # Aspen Early
        (5, 2): (150, 300), # Aspen Mid
        (5, 3): (120, 450), # Aspen Late
    }

    out_raster = Int(-1)  # default: not classified

    for (ftype, stage), (min_stems, min_ba) in thresholds.items():
        # Meets or exceeds both targets
        meets = (
            (forest_type_raster == ftype) &
            (seral_stage_raster == stage) &
            (stand_density_raster >= min_stems) &
            (basal_area_raster >= min_ba)
        )
        # Forest type/seral match, but fails to meet at least one target
        fails = (
            (forest_type_raster == ftype) &
            (seral_stage_raster == stage) &
            ~meets
        )

        out_raster = Con(meets, 1, out_raster)
        out_raster = Con(fails, 0, out_raster)

    return out_raster

# Load rasters
veg_type_r      = Raster("veg_type_tahoe")
seral_stage_r   = Raster("seral_stage_tahoe")
stand_density_r = Raster("stand_density_tahoe")
basal_area_r    = Raster("basal_area_tahoe")

# Build classification and run assessment
forest_type_r = build_forest_type_raster(veg_type_r)
assessment_r  = assess_density_targets(forest_type_r, seral_stage_r, stand_density_r, basal_area_r)

# Save the result
assessment_r.save("stand_density_threshold_assessment")
print("Saved: stand_density_threshold_assessment")


## Seral Stage

In [ ]:
def classify_seral_stage(forest_type_r, seral_stage_r, canopy_cover_r, desired_conditions, output_table="seral_tabulation"):
    # Constants
    cell_size = forest_type_r.meanCellWidth  # Assume square pixels (e.g. 30m x 30m)
    cell_area_m2 = cell_size * cell_size
    m2_to_acres = 1 / 4046.8564224

    # Create canopy class raster (0 = closed, 1 = open)
    canopy_class_r  = Con(canopy_cover_r >= 50, 1, 0)  # For White Fir, Red Fir, etc.
    canopy_class_jp = Con((forest_type_r == 1) & (canopy_cover_r >= 40), 1, 0)  # Jeffrey Pine exception
    canopy_class_r  = Con(forest_type_r == 1, canopy_class_jp, canopy_class_r)

    # Create combined seral+canopy classification raster
    combined_class_r = Con(seral_stage_r == 1, 1,  # Early
                     Con((seral_stage_r == 2) & (canopy_class_r == 0), 2,  # Mid closed
                     Con((seral_stage_r == 2) & (canopy_class_r == 1), 3,  # Mid open
                     Con((seral_stage_r == 3) & (canopy_class_r == 1), 4,  # Late open
                     Con((seral_stage_r == 3) & (canopy_class_r == 0), 5,  # Late closed
                         None)))))

    combined_class_r.save("seral_stage_canopy_combined")

    # Tabulate Area
    TabulateArea(
        in_zone_data=forest_type_r,
        zone_field="Value",
        in_class_data=combined_class_r,
        class_field="Value",
        out_table=output_table,
        processing_cell_size=cell_size
    )

    # Create dataframe from the output table
    fields = ['VALUE', 'VALUE_1', 'VALUE_2', 'VALUE_3', 'VALUE_4', 'VALUE_5']
    data = []

    with arcpy.da.SearchCursor(output_table, fields) as cursor:
        for row in cursor:
            forest_code = row[0]
            if forest_code not in desired_conditions:
                continue  # Skip unknown forest codes
            class_counts = row[1:]
            total_count = sum(class_counts)
            for code, count in enumerate(class_counts, start=1):
                area_m2 = count
                area_acres = area_m2 * m2_to_acres
                percent = round((count / total_count) * 100, 2) if total_count > 0 else 0
                desired_range = desired_conditions[forest_code][code]
                if desired_range[0] <= percent <= desired_range[1]:
                    status = 'on target'
                elif percent < desired_range[0]:
                    status = 'below target'
                else:
                    status = 'above target'
                data.append({
                    "Forest Type Code": forest_code,
                    "Seral Stage Code": code,
                    "Desired % Range": f"{desired_range[0]}–{desired_range[1]}%",
                    "Pixel Count": count,
                    "Area (m²)": round(area_m2, 2),
                    "Area (acres)": round(area_acres, 2),
                    "Current Area %": percent,
                    "Classification": status
                })

    # Add human-readable names
    seral_stage_map = {
        1: "Early",
        2: "Mid (closed canopy)",
        3: "Mid (open canopy)",
        4: "Late (open canopy)",
        5: "Late (closed canopy)"
    }
    veg_type_map = {
        1: "Jeffrey Pine",
        2: "White Fir",
        3: "Red Fir",
        4: "Sub Alpine Conifer",
        5: "Aspen"
    }

    df = pd.DataFrame(data)
    df["Seral Stage"] = df["Seral Stage Code"].map(seral_stage_map)
    df["Forest Type"] = df["Forest Type Code"].map(veg_type_map)
    df = df[["Forest Type", "Seral Stage", "Desired % Range", "Classification", "Current Area %", "Area (m²)", "Area (acres)"]]

    return df

# Define your desired ranges for each forest type and seral class
desired_conditions = {
    1: {1: (5, 15),   2: (5, 10), 3: (25, 30), 4: (45, 50), 5: (5, 10)},    # Jeffrey Pine
    2: {1: (10, 20),  2: (5, 15), 3: (10, 15), 4: (30, 40), 5: (20, 30)},   # White Fir
    3: {1: (10, 20),  2: (20, 30), 3: (5, 15), 4: (15, 25), 5: (25, 35)},   # Red Fir
    4: {1: (10, 20),  2: (10, 20), 3: (5, 15), 4: (15, 25), 5: (10, 20)},   # Sub Alpine Conifer
    5: {1: (10, 20),  2: (5, 15), 3: (10, 20), 4: (15, 25), 5: (25, 35)}    # Aspen
}

# Load rasters
veg_type_r      = Raster("veg_type_tahoe")
seral_stage_r   = Raster("seral_stage_tahoe")
canopy_cover_r  = Raster("canopy_cover_tahoe")
# Run analysis
df = classify_seral_stage(veg_type_r, seral_stage_r, canopy_cover_r, desired_conditions)

# Save to CSV
df.to_csv(output / "seral_stage_with_classification.csv", index=False)


## Functional Fire

##### Dominant Flame Length Class Analysis

In [ ]:
# probability of fire severity 2023
fire_prob_high     = Raster(str(downloads / "ProbabilityHighFireSev_202308/ProbabilityHighFireSev_202308.tif"))
fire_prob_moderate = Raster(str(downloads / "ProbabilityModerateFireSev_202308/ProbabilityModerateFireSev_202308.tif"))
fire_prob_low      = Raster(str(downloads / "ProbabilityLowFireSev_202308/ProbabilityLowFireSev_202308.tif"))

# extract by mask to Tahoe extent
extract_by_mask_to_tahoe_extent(fire_prob_high, "FunctionalFire_HighSeverityProbabiliy_Tahoe_SNVRRK")
extract_by_mask_to_tahoe_extent(fire_prob_moderate, "FunctionalFire_ModerateSeverityProbability_Tahoe_SNVRRK")
extract_by_mask_to_tahoe_extent(fire_prob_low, "FunctionalFire_LowSeverityProbability_Tahoe_SNVRRK")

##### Dominant Class Analysis

In [ ]:
# Load the raw fire severity rasters
highsev_tahoe = Raster("FunctionalFire_HighSeverityProbabiliy_Tahoe_SNVRRK")
modsev_tahoe = Raster("FunctionalFire_ModerateSeverityProbability_Tahoe_SNVRRK")
lowsev_tahoe = Raster("FunctionalFire_LowSeverityProbability_Tahoe_SNVRRK")

# Calculate dominant class
# Use Con and logical operators to determine max
dominant_class = Con(
    (lowsev_tahoe >= modsev_tahoe) & (lowsev_tahoe >= highsev_tahoe), 1,
    Con((modsev_tahoe >= lowsev_tahoe) & (modsev_tahoe >= highsev_tahoe), 2, 3)
)

# Convert to integer raster
dominant_class_int = Raster(Int(dominant_class))

# Save the output raster to the geodatabase
dominant_class_int.save("FunctionalFire_DominantClass_Tahoe_SNVRRK")
print("Exported raster: FunctionalFire_DominantClass_Tahoe_SNVRRK")

# Build an attribute table
arcpy.BuildRasterAttributeTable_management(dominant_class_int, "Overwrite")

# Calculate Acres field
add_acres(dominant_class_int)

# Save the updated raster
dominant_class_int.save("FunctionalFire_DominantClass_Tahoe_SNVRRK")
print("Updated raster: FunctionalFire_DominantClass_Tahoe_SNVRRK")

#### Identify dominant class by management zone

In [ ]:
# export to feature class
arcpy.RasterToPolygon_conversion(
    in_raster="FunctionalFire_DominantClass_Tahoe_SNVRRK",
    out_polygon_features="FunctionalFire_DominantClass_Tahoe_SNVRRK_Polygon",
    simplify="NO_SIMPLIFY",
    raster_field="Value"
)
# add acres field
arcpy.management.AddField("FunctionalFire_DominantClass_Tahoe_SNVRRK_Polygon", "Acres", "DOUBLE")
# calculate acres field
arcpy.management.CalculateField("FunctionalFire_DominantClass_Tahoe_SNVRRK_Polygon", "Acres", "!shape.area@acres!", "PYTHON3")

# identiy with management zones
arcpy.analysis.Identity(
    in_features="FunctionalFire_DominantClass_Tahoe_SNVRRK_Polygon",
    identity_features=str(mgmt_areas),
    out_feature_class="ID_MGMTzones_FunctionalFire_DominantClass_Tahoe_SNVRRK",
)

# calculate acres
arcpy.management.CalculateField("ID_MGMTzones_FunctionalFire_DominantClass_Tahoe_SNVRRK", "Acres", "!shape.area@acres!", "PYTHON3")